In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
from src.llms import get_llms, init_langchain, LLM_GPT_4, LLM_GPT_4O, LLM_GPT_4O_MINI, LLM_SONNET, LLM_GEMINI, LLM_LLAMA
from src.experiment import get_fallacy_identification_df, save_fallacy_identification_df, run_fallacy_identification

# Load environment variables from .env file
load_dotenv()

init_langchain()

In [2]:
df_fallacies = get_fallacy_identification_df()
df_fallacies.head()

[2024-10-16 14:29:02] Loaded existing fallacy identification dataframe from CSV.


,step,entity,fallacy,label,gpt_4o_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,No.
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,No.
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,No.
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,No.
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,No.


In [7]:
llms = get_llms([LLM_GPT_4, LLM_GPT_4O_MINI])
# llms = get_llms([LLM_GPT_4O])

run_fallacy_identification(df_fallacies, llms)

save_fallacy_identification_df(df_fallacies)

Processed 10 responses for LLM gpt_4o.
Processed 20 responses for LLM gpt_4o.
Processed 30 responses for LLM gpt_4o.
Processed 40 responses for LLM gpt_4o.
Processed 50 responses for LLM gpt_4o.
Processed 60 responses for LLM gpt_4o.
Processed 70 responses for LLM gpt_4o.
Processed 80 responses for LLM gpt_4o.
Processed 90 responses for LLM gpt_4o.
Processed 100 responses for LLM gpt_4o.
Processed 110 responses for LLM gpt_4o.
Processed 120 responses for LLM gpt_4o.
Processed 130 responses for LLM gpt_4o.
Processed 140 responses for LLM gpt_4o.
Processed 150 responses for LLM gpt_4o.
Processed 160 responses for LLM gpt_4o.
Processed 170 responses for LLM gpt_4o.
Processed 180 responses for LLM gpt_4o.
Processed 190 responses for LLM gpt_4o.
Processed 200 responses for LLM gpt_4o.
Processed 210 responses for LLM gpt_4o.
Processed 220 responses for LLM gpt_4o.
Processed 230 responses for LLM gpt_4o.
Processed 240 responses for LLM gpt_4o.
Processed 250 responses for LLM gpt_4o.
Processed

In [8]:
df_fallacies.head()

,step,entity,fallacy,label,gpt_4o_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,No.
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,No.
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,No.
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,No.
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,No.


In [9]:
df_fallacies['gpt_4o_response'].value_counts()

gpt_4o_response
No.     2277
Yes.    1581
Yes      779
No         3
Name: count, dtype: int64